In [1]:
import pandas as pd 
import numpy as np 
from optimisation_utiles import * 
from grm_utiles import * 
import matplotlib.pyplot as plt

In [2]:
df_return = pd.read_csv('GRM_returns.csv')

In [3]:
df_return_test = df_return.iloc[-500:].set_index('Date')

In [4]:
%%time
results = portfolio_optimisation(df_return_test, 
                                 target='max_dd',
                                 grm_bound={'first_risk': [0.1, 0.3], 'defensive_factors': [0.1, 0.3], 'trend': [0.1, 0.4], 'tail_risk': [0.1, 0.4]})

Wall time: 19min 26s


In [9]:
results

{'objective': -0.007403137917768979,
 'weights': {'BNPIESTV': 6.472171606348438e-18,
  'BNPIEMTV': 0.007184793760709608,
  'BNPIRPEE': 2.945697777349855e-17,
  'BNPIRPUE': 2.3464409799659693e-17,
  'BNPIRPLE': 1.4461478032319933e-18,
  'BNPIRPJE': 0.004677827567297997,
  'BNPIRPHE': 0.0006881088476444097,
  'BNPIRPAE': 0.0012845069627490005,
  'BNPIV1EE': 0.00033572108403541814,
  'BNPIV1UE': 4.561523971001289e-17,
  'BNPIFU10': 0.08998377693094152,
  'BNPIFU30': 2.667264942048905e-18,
  'BNPXSPPE': 0.09962422123760245,
  'BNPI1X4E': 2.212505812318293e-08,
  'BNPXCFHE': 1.14341331747761e-17,
  'BNPIVOLA': 0.039833452316884815,
  'BNPIFXMG': 0.062147887145579506,
  'BPMMMTWU': 0.0003189965450336887,
  'BNPINMXA': 0.0004782055111148428,
  'BNPITMXA': 0.007971376666941598,
  'BNPXCRME': 0.024911881674574605,
  'BNPXCRMU': 0.08373167226865971,
  'BNPIPQEE': 0.11590097887863862,
  'BNPIPQUE': 0.0032861913363460647,
  'BNPIPRX3': 0.03115484444458743,
  'BNPIDRXA': 0.042108613994868024,
  'BP

In [16]:
((1+df_return_test) * results['weights']).sum(axis=1).cumprod()

Date
2018-12-05    1.000678
2018-12-06    1.004296
2018-12-07    1.003757
2018-12-10    1.004540
2018-12-11    1.003909
                ...   
2020-10-28    1.077702
2020-10-29    1.076157
2020-10-30    1.077288
2020-11-02    1.077566
2020-11-03    1.078071
Length: 500, dtype: float64

In [ ]:
# worst 5% dd MSCI month?
# limit dd by at 25% (GRM +25% of )
# monthly data? 
# plus correlation + contribution to risk 
# rebalance monthly 
# show both one off and rolling 3yrs 

In [ ]:
# drawdown as a constraint, 
# multiple objects? 
# just do monte carlo with monthly data rather than optimiser (with 20bps...)